<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Simple Decision Prompts

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal of prompt engineering is to "tune" the prompt in a way that maximizes the quality and clarity of the AI's response, often using specific wording, context, or examples within the prompt itself.

Decision Intelligence and Prompt Engineering techniques can be combined to created "Generative AI Decision Intellignce"! This can leverage GenAI models to reason over complex tasks, gather intelligence for decisions, recommend decisions, communicate decisions etc. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [7]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [8]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.29.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.29.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Intelligence Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather the Information & Intelligence in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks. 

In [9]:
// New prompt to help with decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

Sure, here are five decision-making frameworks that can enhance the quality of decisions, along with brief descriptions of how each framework supports better analysis and reasoning:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description:** SWOT analysis is a strategic planning tool used to identify the internal and external factors that are favorable and unfavorable to achieving an objective.
   - **Support for Better Decision-Making:** By categorizing the factors into strengths, weaknesses, opportunities, and threats, this framework helps decision-makers systematically analyze the internal and external environment. This balanced assessment facilitates informed choices by identifying what the organization is good at, where it can improve, what external opportunities to leverage, and what threats to mitigate.

2. **Decision Matrix (or Prioritization Matrix)**
   - **Description:** A decision matrix is a tool used to evaluate and prioritize a list of opt

### Step 3 - Execute a Decision Intelligence Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [10]:
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Sure, here are five decision-making frameworks that can enhance the quality of decisions, along with brief descriptions of how each supports better analysis and reasoning in different scenarios:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats):**
   - **Application:** Business strategy, project planning, personal development
   - **How it Helps:** It systematically assesses internal and external factors that can affect a decision. Strengths and weaknesses focus on internal capabilities, whereas opportunities and threats consider external environment. This holistic view helps in strategic planning and risk management.

2. **PDCA Cycle (Plan-Do-Check-Act):**
   - **Application:** Continuous improvement, quality control, project management
   - **How it Helps:** This iterative, four-step management method provides a structured approach to problem-solving. 'Plan' helps in identifying problems and developing hypotheses. 'Do' is about implementing solutions. 'Check' involv

### Step 4 - Execute a Semantic Kernel Decision Prompt with Improved Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can ouput Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting. This table can now be rendered in a Markdown document for easy comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [11]:
// New prompt to help with decision-making frameworks using Markdown formatting.
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

Format the response using a Markdown table. Do not enclose the table in triple backticks.
""";

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptWithMarkdownFormat))
{
   Console.Write(streamChunk);
}

| Framework                  | Description                                                                                                                                                                                                                                                                                                                                |
|----------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis              | SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis helps identify internal strengths and weaknesses, as well as external opportunities and threats. This framework is useful for strategic planning and risk management by providin

| Framework                  | Description                                                                                                                                                                                                                                                                                                                                |
|----------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis              | SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis helps identify internal strengths and weaknesses, as well as external opportunities and threats. This framework is useful for strategic planning and risk management by providing a structured way to think about the pros and cons of different choices.                                         |
| Decision Matrix Analysis   | Also known as a prioritization matrix, this framework helps evaluate and prioritize different options by scoring them against a set of criteria. It is particularly useful in situations where many factors are at play, ensuring a comprehensive evaluation of all available options.                                                                    |
| Cost-Benefit Analysis      | This quantitative approach assesses the financial impacts of different decisions by comparing the expected costs and benefits. It supports better economic reasoning by making explicit the expected monetary outcomes of various alternatives and helps in choosing the option with the highest net benefit.                                        |
| The OODA Loop              | The OODA Loop (Observe, Orient, Decide, Act) is a decision-making process that emphasizes continuous feedback and learning. It is especially effective in rapidly changing environments, such as military strategy or business competition, as it promotes agility and adaptability in decision-making.                                            |
| The Delphi Method          | This structured communication technique uses a panel of experts who answer questionnaires in multiple rounds. After each round, a facilitator provides an anonymous summary of the experts' forecasts. This process helps in arriving at a more rounded and unbiased consensus, particularly useful in complex problem-solving and forecasting.            |

### Step 5 - Execute a Decision Intelligence Prompt with a Custom Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

In [12]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Here are five decision-making frameworks that can enhance the quality of decisions, along with brief descriptions of how each supports better analysis and reasoning:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description**: SWOT Analysis helps in identifying internal and external factors that are favorable and unfavorable to achieve an objective.
   - **Support**: By systematically evaluating strengths, weaknesses, opportunities, and threats, this framework helps decision-makers understand the current situation comprehensively and develop strategies that leverage strengths and opportunities while mitigating weaknesses and threats.

2. **Decision Matrix (or Weighted Scoring Model)**
   - **Description**: A Decision Matrix involves listing options and criteria, assigning weights to each criterion based on importance, and scoring each option against each criterion.
   - **Support**: This framework provides a quantitative approach to comparing m

### Step 6 - Execute a Decision Instelligence Prompt with a System Prompt (Custom AI Persona)

When building prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something wrong
* Provide context (additional information) to the AI
* Using Roles in chat completion prompts
* Give your AI words of encouragement  

One best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a desired persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [16]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply in different scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 750, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Console.WriteLine("Prompt To Send to the Azure OpenAI Chat Completion Service...");
Console.WriteLine(simpleDecisionPromptTemplate);
Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("Response from the Azure OpenAI Chat Completion Service...");


await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Prompt To Send to the Azure OpenAI Chat Completion Service...
System Prompt: You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.

Request from the user: Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply in different scenarios.

----------------------------------------

Response from the Azure OpenAI Chat Completion Service...
Sure! Here are five decision-making frameworks that are easy to understand and can be applied to a variety of daily situations:

1. **Pros and Cons List**:
   - **How it works**: Write down the advantages (pros) and disadvantages (cons) of each option.
   - **Application**: This is useful for straightforward decisions where you can clearly see the benefits and drawbac